Bensimon Mike  
Faustini Benjamin  
Mermier Agathe

# Notre petit Chatbot Jarvis

### Importation des bibliothèques et des données

In [1]:
import json
import pickle
import spacy
import tkinter
from tkinter import *
import time
import random
import numpy as np

import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [2]:
data_file = open('intents.json').read()
intents = json.loads(data_file)

Le j.son est composé de:  
D'un dictionnaire n'ayant qu'une itération (intent:) renvoyant à une liste de dictionnaires.
Dans cette liste, chacun de ses sous-dictionnaires est composé :
- d'un tag: la catégorie de la conversation
- d'un pattern: exemple de phrase que l'utilisateur peut répondre
- des reponses : divers réponses que le chatbot peut utiliser

Le tag 'not-understand' correspond au dictionnaire de réponse du chatbot lorsque qu'il ne comprends pas.

### Préparation des données

In [3]:
nlp = spacy.load('en_core_web_sm') 
word= []
classes = []
documents = []

# On prends chaque phrase du pattern de chaque dictionnaire
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # On tokenise chaque mots
        w = nlp(pattern)
        
        word.extend(w)
        # On associe dans une liste chaque phrase et leur tag
        documents.append((w, intent['tag'])) 
    
    # On ajoute toutes les classes dans la liste classe
    if intent['tag'] not in classes:
        classes.append(intent['tag'])

In [4]:
ignore_words = [',','?','!','-',"'",':'," "]
words=[]
# On lemmatise chaque mots et on enlève les ponctuations.
for i in range(len(word)):
        words.append(word[i].lemma_)

# On crée la liste de mot clé
words = sorted(list(set(words)))
words = [ w for w in words if w not in ignore_words]
classes = sorted(list(set(classes)))

print ("Il y a ", len(documents), "phrases,")
print ("réparties en", len(classes), "classes",":", classes)
print("  ")
print (len(words), "mots clés", words)

Il y a  57 phrases,
réparties en 15 classes : ['Good bye', 'Greetings', 'Thanks', 'age', 'age users', 'citylive', 'day', 'feeling', 'feeling_bad', 'feeling_good', 'hour', 'jokes', 'not_understand', 'options', 'size']
  
82 mots clés ['-PRON-', '24', '26', 'Bye', 'a', 'adio', 'age', 'always', 'amazing', 'an', 'another', 'awesome', 'bad', 'be', 'behind', 'bit', 'bye', 'can', 'class', 'come', 'date', 'day', 'depressed', 'do', 'feel', 'fine', 'for', 'four', 'from', 'generation', 'give', 'good', 'great', 'happy', 'hat', 'have', 'hello', 'help', 'helpfull', 'hey', 'hi', 'how', 'i', 'in', 'joke', 'know', 'laugh', 'life', 'live', 'make', 'morning', 'not', 'okay', 'old', 'on', 'other', 'photo', 'sad', 'see', 'small', 'some', 'someone', 'speak', 'support', 'tall', 'tell', 'thank', 'thanks', 'the', 'there', 'thirty', 'time', 'to', 'today', 'twenty', 'two', 'ty', 'up', 'what', 'where', 'with', 'year']


In [5]:
# Initialisation du dataset
training = []
output_empty = [0] * len(classes)
Lnul = list([0] * len(classes))
for i in range(len(classes)):
    if classes[i] == 'not_understand':
        Lnul[i] = 1
# Pour la partie X
# Pour chaque phrase dans documents
for doc in documents:
    # initialisation du bag
    bag = []
    # On tokenise chaque mot
    pattern_words = doc[0]

    # On les lemmatise
    pattern_words = [ word.lemma_ for word in pattern_words]
    #print(pattern_words)
    
    # On crée le bag de la phrase
    for w in words:
        a =0
        for z in pattern_words:
            if w == z:
                a+=1      
        bag.append(a)

    # Pour Y
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    # On rajoute les 2 à trainning
    training.append([bag, output_row])
print(pattern_words)
print(bag,output_row)
     
# Création des données pour la réponse 'not_understand'
training.append([list([0] * len(words)),Lnul])
training.append([list([0] * len(words)),Lnul])
training.append([list([0] * len(words)),Lnul])
# On mélange et on transforme en np.array
random.shuffle(training)
training = np.array(training)
# Creation du train: X - patterns, Y -Tag
train_x = list(training[:,0])
train_y = list(training[:,1])

['where', 'do', '-PRON-', 'come', 'from', '?']
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0] [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### Entrainement du modèle

In [6]:
# On crée le réseau- 3 couches. La première couche 120 neurones, soit le double du nombre de data,
# la deuxième couche 60 neurones, soit la moitié et la troisième couche sortie contient un nombre de neurones
# égal au nombre d'intentions à prédire avec un softmax
L =len(train_x[0])#120

model = Sequential()
model.add(Dense(L*2, activation='relu', input_dim= L))
model.add(Dropout(0.5))
model.add(Dense( L , activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compilation du modele
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Entrainement du modèle
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)

Epoch 1/100
12/12 [==============================] - 0s 3ms/step - loss: 2.7475 - accuracy: 0.0333
Epoch 2/100
12/12 [==============================] - 0s 1ms/step - loss: 2.6197 - accuracy: 0.0833
Epoch 3/100
12/12 [==============================] - 0s 994us/step - loss: 2.5799 - accuracy: 0.1500
Epoch 4/100
12/12 [==============================] - 0s 1ms/step - loss: 2.5292 - accuracy: 0.2333
Epoch 5/100
12/12 [==============================] - 0s 944us/step - loss: 2.5065 - accuracy: 0.2500
Epoch 6/100
12/12 [==============================] - 0s 968us/step - loss: 2.4569 - accuracy: 0.2333
Epoch 7/100
12/12 [==============================] - 0s 998us/step - loss: 2.4279 - accuracy: 0.2333
Epoch 8/100
12/12 [==============================] - 0s 1ms/step - loss: 2.2771 - accuracy: 0.2667
Epoch 9/100
12/12 [==============================] - 0s 943us/step - loss: 2.2013 - accuracy: 0.3667
Epoch 10/100
12/12 [==============================] - 0s 915us/step - loss: 2.1498 - accuracy: 0.31

12/12 [==============================] - 0s 831us/step - loss: 0.1250 - accuracy: 0.9667
Epoch 82/100
12/12 [==============================] - 0s 748us/step - loss: 0.1715 - accuracy: 0.9500
Epoch 83/100
12/12 [==============================] - 0s 914us/step - loss: 0.1182 - accuracy: 0.9667
Epoch 84/100
12/12 [==============================] - 0s 917us/step - loss: 0.1938 - accuracy: 0.9833
Epoch 85/100
12/12 [==============================] - 0s 1ms/step - loss: 0.0847 - accuracy: 1.0000
Epoch 86/100
12/12 [==============================] - 0s 831us/step - loss: 0.1033 - accuracy: 0.9833
Epoch 87/100
12/12 [==============================] - 0s 748us/step - loss: 0.0811 - accuracy: 1.0000
Epoch 88/100
12/12 [==============================] - 0s 831us/step - loss: 0.0966 - accuracy: 0.9667
Epoch 89/100
12/12 [==============================] - 0s 834us/step - loss: 0.1350 - accuracy: 0.9833
Epoch 90/100
12/12 [==============================] - 0s 748us/step - loss: 0.0529 - accuracy: 1.

### Interface de Jarvis

In [7]:
def bow(sentence, words, show_details=True):
    # On tokenise la phrase de l'utilisateur
    sentence_word = nlp(sentence)
    sentence_words =[]
    # On vérifie si c'est un chiffre puis on lemmatise
    for token in sentence_word:
        if token.is_alpha == False:
            sentence_words.append( '24')
        else:
            sentence_words.append(token.lemma_)
    # On fait le bag correspondant à la phrase de l'utilisateur
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    p = bow(sentence, words,show_details=False)
    #On entre le bag dans le réseau
    res = model.predict(np.array([p]))[0]
    #print(res)
    # On recupère la prédiction la plus haute au dessus de 0.3
    ERROR_THRESHOLD = 0.3
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    # On recupère le tag et la probabilité de sa prédiction
    return return_list

def chatbot_response(msg):
    ints = predict_class(msg, model)
    # On récupère le tag de la phrase de l'utilisateur
    tag = ints[0]['intent']
    list_of_intents = intents['intents']
    # On cherche une réponse correspondant au tag
    for i in list_of_intents:
        if(i['tag']== tag):
            res = random.choice(i['responses'])
            break
    return res,tag

In [8]:
# Creation d'une fenêtre pop-up avec tkinter
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        now = time.localtime(time.time())
        year, month, day, hour, minute, second, weekday, yearday, daylight = now
        res, tag = chatbot_response(msg)
        # Si le tag est 'hour' pour afficher l'heure
        if tag =='hour':
            ChatLog.config(state=NORMAL)
            ChatLog.insert(END, "You: " + msg + '\n\n')
            ChatLog.config(foreground="#3b1cd6", font=("Verdana", 12 ))
            ChatLog.config(state=NORMAL)            
            ChatLog.insert(END, "Jarvis : {} : {}: {}"'\n\n'.format(hour, minute, second))
            ChatLog.insert(END, "     " + res + '\n\n')
            ChatLog.config(state=DISABLED)
            ChatLog.yview(END)
        # Si le tag est 'day' pour afficher le jour
        elif tag == 'day':
            ChatLog.config(state=NORMAL)
            ChatLog.insert(END, "You: " + msg + '\n\n')
            ChatLog.config(foreground="#3b1cd6", font=("Verdana", 12 ))
            ChatLog.config(state=NORMAL)            
            ChatLog.insert(END, "Jarvis : {}/{}/{}"'\n\n'.format(day, month, year))
            ChatLog.insert(END, "     " + res + '\n\n')
            ChatLog.config(state=DISABLED)
            ChatLog.yview(END)
        else:
            ChatLog.config(state=NORMAL)
            ChatLog.insert(END, "You: " + msg + '\n\n')
            ChatLog.config(foreground="#3b1cd6", font=("Verdana", 12 ))
            ChatLog.config(state=NORMAL)
            ChatLog.insert(END, "Jarvis : " + res + '\n\n')
            ChatLog.config(state=DISABLED)
            ChatLog.yview(END)

base = Tk()
base.title("Our little Chatbot")
base.geometry("500x600")
base.resizable(width=FALSE, height=FALSE)
# Creation de la fenêtre du Chat
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)
ChatLog.config(state=DISABLED)
# Scrollbar 
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set
#Boutton 'send'
SendButton = Button(base, font=("blue",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#3c3f9d", activebackground="#05eaff",fg='#ffffff',
                    command= send )
# Boite d'écriture des messages
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
# On place les différents éléments dans la fenêtre
scrollbar.place(x=470,y=6, height=586)
ChatLog.place(x=6,y=6, height=470, width=460)
EntryBox.place(x=128, y=480, height=120, width=340)
SendButton.place(x=1, y=480, height=120)

base.mainloop()